# 8. Figures

## Heatmaps

In [ ]:
population_obs = 'population' #Could be any obs
populations_to_plot = adata.obs[population_obs].cat.categories.tolist() #This will plot ALL the population in that .obs, but you can instead supply a list of pops
markers_to_plot = adata.var_names #This will plot ALL the markers, but you can instead supply a list of markers

##############################

figure_dir=Path('Figures','Heatmaps')
os.makedirs(figure_dir, exist_ok=True)

adata_plotting = adata[adata.obs[population_obs].isin(populations_to_plot)].copy()

sc.tl.dendrogram(adata_plotting, groupby=population_obs)

fig = sc.pl.matrixplot(adata_plotting,
                        var_names= markers_to_plot,
                        groupby=population_obs, 
                        vmax=0.6, #Change the vmax here
                        dendrogram=True,
                        return_fig=True)

fig.savefig(Path(figure_dir, f'{population_obs}_heatmap.png'), bbox_inches='tight', dpi=300)